In [1]:
#numpy loader method to load numpy arrays into the Pytorch ImageFolder dataset
import numpy as np

def my_numpy_loader(filename):
    return np.load(filename)

In [2]:
import os
os.chdir("../augmented_32x32x48")
print(os.getcwd())



D:\github\augmented_32x32x48


In [6]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import visdom


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#Visdom Configuration    
viz = visdom.Visdom()

# Hyper parameters
num_epochs = 100
num_classes = 2
batch_size = 100
learning_rate = 0.001

# Dicom dataset
train_dataset = torchvision.datasets.DatasetFolder(root='2D/train', 
                                           transform=transforms.ToTensor(), loader=my_numpy_loader, extensions="npy")

test_dataset = torchvision.datasets.DatasetFolder(root='2D/test', 
                                          transform=transforms.ToTensor(), loader=my_numpy_loader, extensions="npy")
print(train_dataset)
print(test_dataset)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet,self).__init__()
        #Convo 1
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.batch_norm1 = nn.BatchNorm2d(16)
        self.relu1=nn.ReLU()
        #Maxpool_1
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        #Convo_2
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.relu2=nn.ReLU()
        #Maxpool_2
        self.maxpool2=nn.MaxPool2d(kernel_size=2)

        #Linear Layer
        self.fc1=nn.Linear(2048,1024)
        self.fc2=nn.Linear(1024,32)
        self.fc3=nn.Linear(32,2)

    def forward(self,x):
        #Convo_1
        out=self.cnn1(x)
        #print(out.shape)
        out=self.batch_norm1(out)
        #print(out.shape)
        out=self.relu1(out)
        #print(out.shape)
        
        #Max_pool1
        out=self.maxpool1(out)
        #print(out.shape)
        
        #Convo_2
        out=self.cnn2(out)
        #print(out.shape)
        out=self.batch_norm2(out)
        #print(out.shape)
        out=self.relu2(out)
        #print(out.shape)
        
        #Max_pool2
        out=self.maxpool2(out)
        #print(out.shape)
        out=out.view(out.size(0),-1)#Flattening out 
        #print(out.shape)
        
        #Linear Layers
        out=self.fc1(out)
        #print(out.shape)
        out=self.fc2(out)
        #print(out.shape)
        out=self.fc3(out)
        #print(out.shape)
        return out

model = ConvNet(num_classes).to(device).double()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
loss_window = viz.line(
    Y=torch.zeros((1)).cpu(),
    X=torch.zeros((1)).cpu(),
    opts=dict(xlabel='Epoch',ylabel='Loss',title='2D Training Loss'))

accuracy_window = viz.line(
    Y=torch.zeros((1)).cpu(),
    X=torch.zeros((1)).cpu(),
    opts=dict(xlabel='Epoch',ylabel='Accuracy [%]',title='2D Network Accuracy'))


counter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        loss_array = np.array([loss.item()])
        counter = counter + 1

    # Test the model
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        incorrect = 0
        total = 0
        true_positive = 0
        false_negative = 0
        true_negative = 0
        false_positive = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        print('Test Accuracy of the model on the 212 test images: {} %'.format(100 * correct / total))
        acc = 100 * correct / total
    viz.line(X=torch.ones((1,1)).cpu()*epoch, Y=torch.Tensor([loss]).unsqueeze(0),win=loss_window,update='append')
        
    viz.line(X=torch.ones((1,1)).cpu()*epoch, Y=torch.Tensor([acc]).unsqueeze(0),win=accuracy_window,update='append')

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')

Dataset DatasetFolder
    Number of datapoints: 1964
    Root Location: 2D/train
    Transforms (if any): ToTensor()
    Target Transforms (if any): None
Dataset DatasetFolder
    Number of datapoints: 844
    Root Location: 2D/test
    Transforms (if any): ToTensor()
    Target Transforms (if any): None
Epoch [1/100], Step [1/20], Loss: 0.6672
Epoch [1/100], Step [2/20], Loss: 4.2278
Epoch [1/100], Step [3/20], Loss: 1.2814
Epoch [1/100], Step [4/20], Loss: 2.0293
Epoch [1/100], Step [5/20], Loss: 1.8149
Epoch [1/100], Step [6/20], Loss: 0.5014
Epoch [1/100], Step [7/20], Loss: 1.3967
Epoch [1/100], Step [8/20], Loss: 1.5385
Epoch [1/100], Step [9/20], Loss: 0.9335
Epoch [1/100], Step [10/20], Loss: 0.4915
Epoch [1/100], Step [11/20], Loss: 0.5796
Epoch [1/100], Step [12/20], Loss: 0.6343
Epoch [1/100], Step [13/20], Loss: 0.7143
Epoch [1/100], Step [14/20], Loss: 0.5939
Epoch [1/100], Step [15/20], Loss: 0.4639
Epoch [1/100], Step [16/20], Loss: 0.4007
Epoch [1/100], Step [17/20], Lo

Epoch [9/100], Step [18/20], Loss: 0.1068
Epoch [9/100], Step [19/20], Loss: 0.2620
Epoch [9/100], Step [20/20], Loss: 0.2242
Test Accuracy of the model on the 212 test images: 93.48341232227489 %
Epoch [10/100], Step [1/20], Loss: 0.1302
Epoch [10/100], Step [2/20], Loss: 0.1721
Epoch [10/100], Step [3/20], Loss: 0.1850
Epoch [10/100], Step [4/20], Loss: 0.1587
Epoch [10/100], Step [5/20], Loss: 0.2247
Epoch [10/100], Step [6/20], Loss: 0.1151
Epoch [10/100], Step [7/20], Loss: 0.1796
Epoch [10/100], Step [8/20], Loss: 0.1433
Epoch [10/100], Step [9/20], Loss: 0.0807
Epoch [10/100], Step [10/20], Loss: 0.2200
Epoch [10/100], Step [11/20], Loss: 0.2439
Epoch [10/100], Step [12/20], Loss: 0.1568
Epoch [10/100], Step [13/20], Loss: 0.3092
Epoch [10/100], Step [14/20], Loss: 0.1099
Epoch [10/100], Step [15/20], Loss: 0.1313
Epoch [10/100], Step [16/20], Loss: 0.2071
Epoch [10/100], Step [17/20], Loss: 0.1322
Epoch [10/100], Step [18/20], Loss: 0.1301
Epoch [10/100], Step [19/20], Loss: 0.

Epoch [18/100], Step [19/20], Loss: 0.1825
Epoch [18/100], Step [20/20], Loss: 0.1331
Test Accuracy of the model on the 212 test images: 95.14218009478672 %
Epoch [19/100], Step [1/20], Loss: 0.1187
Epoch [19/100], Step [2/20], Loss: 0.0954
Epoch [19/100], Step [3/20], Loss: 0.1514
Epoch [19/100], Step [4/20], Loss: 0.2898
Epoch [19/100], Step [5/20], Loss: 0.1572
Epoch [19/100], Step [6/20], Loss: 0.0504
Epoch [19/100], Step [7/20], Loss: 0.1524
Epoch [19/100], Step [8/20], Loss: 0.1810
Epoch [19/100], Step [9/20], Loss: 0.0799
Epoch [19/100], Step [10/20], Loss: 0.0362
Epoch [19/100], Step [11/20], Loss: 0.1842
Epoch [19/100], Step [12/20], Loss: 0.0618
Epoch [19/100], Step [13/20], Loss: 0.1601
Epoch [19/100], Step [14/20], Loss: 0.1368
Epoch [19/100], Step [15/20], Loss: 0.2340
Epoch [19/100], Step [16/20], Loss: 0.0622
Epoch [19/100], Step [17/20], Loss: 0.1267
Epoch [19/100], Step [18/20], Loss: 0.1704
Epoch [19/100], Step [19/20], Loss: 0.1314
Epoch [19/100], Step [20/20], Loss:

Epoch [27/100], Step [19/20], Loss: 0.1407
Epoch [27/100], Step [20/20], Loss: 0.0734
Test Accuracy of the model on the 212 test images: 96.44549763033176 %
Epoch [28/100], Step [1/20], Loss: 0.0568
Epoch [28/100], Step [2/20], Loss: 0.0852
Epoch [28/100], Step [3/20], Loss: 0.0483
Epoch [28/100], Step [4/20], Loss: 0.0433
Epoch [28/100], Step [5/20], Loss: 0.1116
Epoch [28/100], Step [6/20], Loss: 0.0816
Epoch [28/100], Step [7/20], Loss: 0.2391
Epoch [28/100], Step [8/20], Loss: 0.0924
Epoch [28/100], Step [9/20], Loss: 0.0389
Epoch [28/100], Step [10/20], Loss: 0.0814
Epoch [28/100], Step [11/20], Loss: 0.0715
Epoch [28/100], Step [12/20], Loss: 0.1777
Epoch [28/100], Step [13/20], Loss: 0.0855
Epoch [28/100], Step [14/20], Loss: 0.0672
Epoch [28/100], Step [15/20], Loss: 0.1592
Epoch [28/100], Step [16/20], Loss: 0.1194
Epoch [28/100], Step [17/20], Loss: 0.1169
Epoch [28/100], Step [18/20], Loss: 0.1178
Epoch [28/100], Step [19/20], Loss: 0.0884
Epoch [28/100], Step [20/20], Loss:

Epoch [36/100], Step [18/20], Loss: 0.0246
Epoch [36/100], Step [19/20], Loss: 0.0387
Epoch [36/100], Step [20/20], Loss: 0.0144
Test Accuracy of the model on the 212 test images: 97.27488151658768 %
Epoch [37/100], Step [1/20], Loss: 0.0634
Epoch [37/100], Step [2/20], Loss: 0.1498
Epoch [37/100], Step [3/20], Loss: 0.0321
Epoch [37/100], Step [4/20], Loss: 0.0436
Epoch [37/100], Step [5/20], Loss: 0.0827
Epoch [37/100], Step [6/20], Loss: 0.0220
Epoch [37/100], Step [7/20], Loss: 0.0594
Epoch [37/100], Step [8/20], Loss: 0.0132
Epoch [37/100], Step [9/20], Loss: 0.1101
Epoch [37/100], Step [10/20], Loss: 0.0456
Epoch [37/100], Step [11/20], Loss: 0.0486
Epoch [37/100], Step [12/20], Loss: 0.0517
Epoch [37/100], Step [13/20], Loss: 0.0695
Epoch [37/100], Step [14/20], Loss: 0.0945
Epoch [37/100], Step [15/20], Loss: 0.0859
Epoch [37/100], Step [16/20], Loss: 0.0759
Epoch [37/100], Step [17/20], Loss: 0.0701
Epoch [37/100], Step [18/20], Loss: 0.0342
Epoch [37/100], Step [19/20], Loss:

...

Epoch [63/100], Step [17/20], Loss: 0.0008
Epoch [63/100], Step [18/20], Loss: 0.0010
Epoch [63/100], Step [19/20], Loss: 0.0022
Epoch [63/100], Step [20/20], Loss: 0.0031
Test Accuracy of the model on the 212 test images: 99.28909952606635 %
Epoch [64/100], Step [1/20], Loss: 0.0005
Epoch [64/100], Step [2/20], Loss: 0.0028
Epoch [64/100], Step [3/20], Loss: 0.0005
Epoch [64/100], Step [4/20], Loss: 0.0017
Epoch [64/100], Step [5/20], Loss: 0.0003
Epoch [64/100], Step [6/20], Loss: 0.0005
Epoch [64/100], Step [7/20], Loss: 0.0040
Epoch [64/100], Step [8/20], Loss: 0.0004
Epoch [64/100], Step [9/20], Loss: 0.0011
Epoch [64/100], Step [10/20], Loss: 0.0003
Epoch [64/100], Step [11/20], Loss: 0.0007
Epoch [64/100], Step [12/20], Loss: 0.0016
Epoch [64/100], Step [13/20], Loss: 0.0011
Epoch [64/100], Step [14/20], Loss: 0.0003
Epoch [64/100], Step [15/20], Loss: 0.0009
Epoch [64/100], Step [16/20], Loss: 0.0008
Epoch [64/100], Step [17/20], Loss: 0.0009
Epoch [64/100], Step [18/20], Loss:

Epoch [72/100], Step [15/20], Loss: 0.0004
Epoch [72/100], Step [16/20], Loss: 0.0005
Epoch [72/100], Step [17/20], Loss: 0.0003
Epoch [72/100], Step [18/20], Loss: 0.0004
Epoch [72/100], Step [19/20], Loss: 0.0009
Epoch [72/100], Step [20/20], Loss: 0.0009
Test Accuracy of the model on the 212 test images: 99.28909952606635 %
Epoch [73/100], Step [1/20], Loss: 0.0009
Epoch [73/100], Step [2/20], Loss: 0.0005
Epoch [73/100], Step [3/20], Loss: 0.0004
Epoch [73/100], Step [4/20], Loss: 0.0004
Epoch [73/100], Step [5/20], Loss: 0.0002
Epoch [73/100], Step [6/20], Loss: 0.0006
Epoch [73/100], Step [7/20], Loss: 0.0002
Epoch [73/100], Step [8/20], Loss: 0.0003
Epoch [73/100], Step [9/20], Loss: 0.0004
Epoch [73/100], Step [10/20], Loss: 0.0003
Epoch [73/100], Step [11/20], Loss: 0.0003
Epoch [73/100], Step [12/20], Loss: 0.0002
Epoch [73/100], Step [13/20], Loss: 0.0006
Epoch [73/100], Step [14/20], Loss: 0.0004
Epoch [73/100], Step [15/20], Loss: 0.0008
Epoch [73/100], Step [16/20], Loss:

Epoch [81/100], Step [15/20], Loss: 0.0003
Epoch [81/100], Step [16/20], Loss: 0.0003
Epoch [81/100], Step [17/20], Loss: 0.0001
Epoch [81/100], Step [18/20], Loss: 0.0005
Epoch [81/100], Step [19/20], Loss: 0.0001
Epoch [81/100], Step [20/20], Loss: 0.0001
Test Accuracy of the model on the 212 test images: 99.28909952606635 %
Epoch [82/100], Step [1/20], Loss: 0.0002
Epoch [82/100], Step [2/20], Loss: 0.0002
Epoch [82/100], Step [3/20], Loss: 0.0001
Epoch [82/100], Step [4/20], Loss: 0.0002
Epoch [82/100], Step [5/20], Loss: 0.0003
Epoch [82/100], Step [6/20], Loss: 0.0001
Epoch [82/100], Step [7/20], Loss: 0.0001
Epoch [82/100], Step [8/20], Loss: 0.0001
Epoch [82/100], Step [9/20], Loss: 0.0010
Epoch [82/100], Step [10/20], Loss: 0.0001
Epoch [82/100], Step [11/20], Loss: 0.0002
Epoch [82/100], Step [12/20], Loss: 0.0001
Epoch [82/100], Step [13/20], Loss: 0.0003
Epoch [82/100], Step [14/20], Loss: 0.0003
Epoch [82/100], Step [15/20], Loss: 0.0002
Epoch [82/100], Step [16/20], Loss:

Epoch [90/100], Step [15/20], Loss: 0.0003
Epoch [90/100], Step [16/20], Loss: 0.0002
Epoch [90/100], Step [17/20], Loss: 0.0001
Epoch [90/100], Step [18/20], Loss: 0.0001
Epoch [90/100], Step [19/20], Loss: 0.0001
Epoch [90/100], Step [20/20], Loss: 0.0001
Test Accuracy of the model on the 212 test images: 99.28909952606635 %
Epoch [91/100], Step [1/20], Loss: 0.0002
Epoch [91/100], Step [2/20], Loss: 0.0001
Epoch [91/100], Step [3/20], Loss: 0.0002
Epoch [91/100], Step [4/20], Loss: 0.0001
Epoch [91/100], Step [5/20], Loss: 0.0002
Epoch [91/100], Step [6/20], Loss: 0.0002
Epoch [91/100], Step [7/20], Loss: 0.0002
Epoch [91/100], Step [8/20], Loss: 0.0001
Epoch [91/100], Step [9/20], Loss: 0.0003
Epoch [91/100], Step [10/20], Loss: 0.0001
Epoch [91/100], Step [11/20], Loss: 0.0000
Epoch [91/100], Step [12/20], Loss: 0.0002
Epoch [91/100], Step [13/20], Loss: 0.0000
Epoch [91/100], Step [14/20], Loss: 0.0003
Epoch [91/100], Step [15/20], Loss: 0.0003
Epoch [91/100], Step [16/20], Loss:

Epoch [99/100], Step [15/20], Loss: 0.0001
Epoch [99/100], Step [16/20], Loss: 0.0001
Epoch [99/100], Step [17/20], Loss: 0.0002
Epoch [99/100], Step [18/20], Loss: 0.0000
Epoch [99/100], Step [19/20], Loss: 0.0000
Epoch [99/100], Step [20/20], Loss: 0.0003
Test Accuracy of the model on the 212 test images: 99.28909952606635 %
Epoch [100/100], Step [1/20], Loss: 0.0001
Epoch [100/100], Step [2/20], Loss: 0.0001
Epoch [100/100], Step [3/20], Loss: 0.0001
Epoch [100/100], Step [4/20], Loss: 0.0001
Epoch [100/100], Step [5/20], Loss: 0.0002
Epoch [100/100], Step [6/20], Loss: 0.0001
Epoch [100/100], Step [7/20], Loss: 0.0000
Epoch [100/100], Step [8/20], Loss: 0.0001
Epoch [100/100], Step [9/20], Loss: 0.0002
Epoch [100/100], Step [10/20], Loss: 0.0001
Epoch [100/100], Step [11/20], Loss: 0.0001
Epoch [100/100], Step [12/20], Loss: 0.0001
Epoch [100/100], Step [13/20], Loss: 0.0001
Epoch [100/100], Step [14/20], Loss: 0.0002
Epoch [100/100], Step [15/20], Loss: 0.0002
Epoch [100/100], Ste